## 日内分钟级数据的CVaR因子

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
tqdm.pandas()
# pd.set_option('display.max_rows', 50)

/mnt/datadisk2/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def cVaR_series(series, c_level=0.9, is_low=True):
    '''
    CVaR:分布的尾部积分, Parameters:
    c_level:p值,默认0.9
    is_low:返回的是否是左侧CVaR,=False时返回右侧,默认True
    '''
    right_VaR = series.quantile(c_level)
    left_VaR = series.quantile(1- c_level)

    if is_low:
        X_low = pd.Series(series[series < left_VaR])
        low_mean = X_low.mean() if len(X_low) > 0 else 0
        return low_mean
    else:
        X_high = pd.Series(series[series > right_VaR])
        high_mean = X_high.mean() if len(X_high) > 0 else 0
        return -high_mean

In [3]:
open2021 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/open_2021.h5')
open2021 = open2021.set_index(['Date', 'EndTime'])
open2021

InstrumentID      SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20210104 09:31        9.64     14.10      8.95     12.45      2.86     75.70   
         09:32        9.64     14.01      8.82     12.35      2.86     76.46   
         09:33        9.60     13.97      8.84     12.36      2.84     76.17   
         09:34        9.60     14.00      8.83     12.37      2.85     76.10   
         09:35        9.58     13.90      8.83     12.33      2.85     75.74   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56        8.55     12.04      6.81     14.41      3.41     46.68   
         14:57        8.54     12.03      6.81     14.41      3.40     46.68   
         14:58        8.55     12.03      6.81     14.39      3.41     46.70   
         14:59        8.55     12.03      6.81     14.39      3.41     46.70   
         15:00        8.55     12.03      6.81     14.39      3.41     46.70   

InstrumentID      SH600010  SH600011  SH600012  SH600015  ...  SZ301190  \
Date     EndTime                                          ...             
20210104 09:31        1.17      4.50      6.14      6.22  ...       NaN   
         09:32        1.18      4.49      6.12      6.22  ...       NaN   
         09:33        1.17      4.47      6.16      6.20  ...       NaN   
         09:34        1.17      4.47      6.17      6.21  ...       NaN   
         09:35        1.17      4.46      6.16      6.21  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20211231 14:56        2.78      9.64      7.07      5.59  ...     30.31   
         14:57        2.79      9.65      7.07      5.59  ...     30.28   
         14:58        2.79      9.65      7.06      5.60  ...     30.20   
         14:59        2.79      9.65      7.06      5.60  ...     30.20   
         15:00        2.79      9.65      7.06      5.60  ...     30.20   

InstrumentID      SH688210  SH688206  SH688265  SZ301166  SZ301189  SH688227  \
Date     EndTime                                                               
20210104 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20211231 14:56       41.48     36.95     69.92    104.04     60.49     32.14   
         14:57       41.51     36.93     70.00    103.70     60.32     32.12   
         14:58       41.48     36.73     70.05    103.20     60.05     32.09   
         14:59       41.48     36.73     70.05    103.30     60.02     32.09   
         15:00       41.48     36.73     70.05    103.30     60.02     32.09   

InstrumentID      SH688236  SZ301127  SH603176  
Date     EndTime                                
20210104 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20211231 14:56       27.39     20.23      2.45  
         14:57       27.40     20.30      2.45  
         14:58       27.39     20.25      2.45  
         14:59       27.39     20.19      2.45  
         15:00       27.39     20.19      2.45  

[58320 rows x 4622 columns]

In [3]:
def min_data_cVaR(price:pd.DataFrame):
    def _min_data_cVaR(price_daily:pd.DataFrame):
        return_daily = price_daily.pct_change().fillna(0)
        
        return return_daily.apply(cVaR_series)
    
    return price.groupby(level='Date').progress_apply(_min_data_cVaR)

In [7]:
f1 = min_data_cVaR(open2021)
f1

InstrumentID  SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                       
20210104     -0.002489 -0.002604 -0.005117 -0.002957 -0.006110 -0.003075   
20210105     -0.001675 -0.002426 -0.006855 -0.001922 -0.004014 -0.002728   
20210106     -0.001678 -0.002765 -0.005206 -0.002439 -0.003835 -0.002591   
20210107     -0.002400 -0.003025 -0.006216 -0.003578 -0.003906 -0.002387   
20210108     -0.002419 -0.002861 -0.005919 -0.002562 -0.003690 -0.002498   
...                ...       ...       ...       ...       ...       ...   
20211227     -0.001373 -0.002288 -0.001797 -0.002773 -0.004420 -0.001909   
20211228     -0.001472 -0.002883 -0.001591 -0.002156 -0.004972 -0.002129   
20211229     -0.001870 -0.001966 -0.001919 -0.001751 -0.003259 -0.001980   
20211230     -0.002335 -0.002137 -0.001465 -0.001694 -0.003968 -0.001801   
20211231     -0.001171 -0.002112 -0.002440 -0.001972 -0.003267 -0.001836   

InstrumentID  SH600010  SH600011  SH600012  SH600015  ...  SZ301190  SH688210  \
Date                                                  ...                       
20210104     -0.008547 -0.003470 -0.002889 -0.002897  ...  0.000000  0.000000   
20210105      0.000000 -0.003102 -0.003773 -0.002218  ...  0.000000  0.000000   
20210106      0.000000 -0.002888 -0.004765 -0.001840  ...  0.000000  0.000000   
20210107     -0.008547 -0.003338 -0.003229 -0.002272  ...  0.000000  0.000000   
20210108      0.000000 -0.003215 -0.004016 -0.002475  ...  0.000000  0.000000   
...                ...       ...       ...       ...  ...       ...       ...   
20211227     -0.005553 -0.007708 -0.002072  0.000000  ... -0.015806 -0.009663   
20211228     -0.004163 -0.006616 -0.002195  0.000000  ... -0.005552 -0.008154   
20211229     -0.004474 -0.005607 -0.001945 -0.001789  ... -0.008270 -0.003561   
20211230     -0.003748 -0.004607 -0.001671 -0.001916  ... -0.007440 -0.003635   
20211231     -0.003936 -0.005091 -0.001954 -0.001789  ... -0.004455 -0.004019   

InstrumentID  SH688206  SH688265  SZ301166  SZ301189  SH688227  SH688236  \
Date                                                                       
20210104      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20210105      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20210106      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20210107      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20210108      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...                ...       ...       ...       ...       ...       ...   
20211227      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20211228     -0.016309 -0.010780 -0.019425 -0.019387  0.000000  0.000000   
20211229     -0.007330 -0.005403 -0.015898 -0.007706  0.000000  0.000000   
20211230     -0.006240 -0.006047 -0.013939 -0.010074 -0.010527 -0.012093   
20211231     -0.005027 -0.003775 -0.008419 -0.008479 -0.004510 -0.004468   

InstrumentID  SZ301127  SH603176  
Date                              
20210104      0.000000       0.0  
20210105      0.000000       0.0  
20210106      0.000000       0.0  
20210107      0.000000       0.0  
20210108      0.000000       0.0  
...                ...       ...  
20211227      0.000000       0.0  
20211228      0.000000       0.0  
20211229      0.000000       0.0  
20211230     -0.018765       0.0  
20211231     -0.012275       0.0  

[243 rows x 4622 columns]

In [4]:
# f1.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor_test/fa_21_23/interday_cvar_neg_0.9_min.csv')

In [5]:
data2023 = pd.read_hdf('/mnt/datadisk2/aglv/aglv/lab_aglv/data2023.h5')
data2023

Open  Close   High    Low  LastVolume  \
Date     InstrumentID EndTime                                           
20230103 SZ000001     09:31    13.20  13.16  13.23  13.06   3725537.0   
                      09:32    13.16  13.14  13.18  13.11   1122500.0   
                      09:33    13.14  13.17  13.17  13.11   1284694.0   
                      09:34    13.17  13.20  13.20  13.16   1218457.0   
                      09:35    13.20  13.19  13.19  13.16   1066087.0   
...                              ...    ...    ...    ...         ...   
20231012 SH689009     14:56    33.94  33.88  33.94  33.88     23275.0   
                      14:57    33.92  33.89  33.92  33.86      9297.0   
                      14:58    33.89  33.89  33.89  33.89         0.0   
                      14:59    33.89  33.89  33.89  33.89         0.0   
                      15:00    33.77  33.77  33.77  33.77     62011.0   

                               LastTurnOver  
Date     InstrumentID EndTime                
20230103 SZ000001     09:31    4.894731e+07  
                      09:32    1.473833e+07  
                      09:33    1.688369e+07  
                      09:34    1.605823e+07  
                      09:35    1.404670e+07  
...                                     ...  
20231012 SH689009     14:56    7.893110e+05  
                      14:57    3.150930e+05  
                      14:58    0.000000e+00  
                      14:59    0.000000e+00  
                      15:00    2.094111e+06  

[222608001 rows x 6 columns]

In [7]:
open2023 = data2023['Open'].unstack(level='InstrumentID')
open2023

InstrumentID      SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date     EndTime                                                               
20230103 09:31        7.27     14.99      5.58     15.75      2.84     57.56   
         09:32        7.22     14.91      5.54     15.78      2.83     57.46   
         09:33        7.23     14.92      5.53     15.82      2.83     57.39   
         09:34        7.24     14.87      5.57     15.87      2.84     57.31   
         09:35        7.25     14.99      5.56     15.80      2.83     57.48   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56        7.13     10.87      5.79     19.03      2.82     37.33   
         14:57        7.13     10.87      5.78     18.98      2.81     37.33   
         14:58        7.13     10.87      5.79     19.04      2.82     37.33   
         14:59        7.13     10.87      5.79     19.04      2.82     37.33   
         15:00        7.14     10.88      5.79     18.98      2.81     37.33   

InstrumentID      SH600010  SH600011  SH600012  SH600015  ...  SZ301519  \
Date     EndTime                                          ...             
20230103 09:31        1.92      7.70      7.30      5.20  ...       NaN   
         09:32        1.92      7.61      7.32      5.18  ...       NaN   
         09:33        1.93      7.63      7.33      5.18  ...       NaN   
         09:34        1.94      7.67      7.33      5.18  ...       NaN   
         09:35        1.94      7.68      7.32      5.18  ...       NaN   
...                    ...       ...       ...       ...  ...       ...   
20231012 14:56        1.73      7.82     10.84      5.72  ...     23.04   
         14:57        1.72      7.80     10.84      5.71  ...     23.06   
         14:58        1.73      7.82     10.81      5.73  ...     23.06   
         14:59        1.73      7.82     10.81      5.73  ...     23.06   
         15:00        1.72      7.81     10.83      5.71  ...     23.05   

InstrumentID      SZ301520  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  \
Date     EndTime                                                               
20230103 09:31         NaN       NaN       NaN       NaN       NaN       NaN   
         09:32         NaN       NaN       NaN       NaN       NaN       NaN   
         09:33         NaN       NaN       NaN       NaN       NaN       NaN   
         09:34         NaN       NaN       NaN       NaN       NaN       NaN   
         09:35         NaN       NaN       NaN       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
20231012 14:56       65.80    102.10     84.05     44.23     41.44     61.35   
         14:57       65.85    101.96     84.10     44.24     41.44     61.35   
         14:58       65.64    101.81     84.10     44.28     41.39     61.34   
         14:59       65.64    101.81     84.10     44.28     41.39     61.34   
         15:00       65.64    101.90     84.29     44.35     41.40     61.33   

InstrumentID      SZ301550  SZ301558  SZ301559  
Date     EndTime                                
20230103 09:31         NaN       NaN       NaN  
         09:32         NaN       NaN       NaN  
         09:33         NaN       NaN       NaN  
         09:34         NaN       NaN       NaN  
         09:35         NaN       NaN       NaN  
...                    ...       ...       ...  
20231012 14:56       50.62     14.10     22.73  
         14:57       50.38     14.08     22.72  
         14:58       50.18     14.00     22.69  
         14:59       50.18     14.00     22.69  
         15:00       50.30     14.01     22.69  

[44718 rows x 5111 columns]

In [8]:
cvar_neg = min_data_cVaR(open2023)
cvar_neg

InstrumentID  SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                       
20230103     -0.002256 -0.002980 -0.002242 -0.002562 -0.003519 -0.002031   
20230104     -0.001712 -0.001748 -0.001859 -0.002272 -0.003460 -0.001781   
20230105     -0.002010 -0.002385 -0.002014 -0.002472 -0.004489 -0.002062   
20230106     -0.001772 -0.002407 -0.001874 -0.003111 -0.003474 -0.002270   
20230109     -0.002511 -0.002597 -0.002282 -0.002461 -0.003688 -0.001811   
...                ...       ...       ...       ...       ...       ...   
20230928     -0.001666 -0.001611 -0.001812 -0.001356  0.000000 -0.001129   
20231009     -0.001954 -0.002545 -0.002074 -0.002048 -0.003546 -0.001756   
20231010     -0.002128 -0.001681 -0.002172 -0.001428 -0.003559 -0.001214   
20231011     -0.001973 -0.001394 -0.002257 -0.001707 -0.003559 -0.001009   
20231012     -0.001901 -0.001124 -0.002033 -0.001481  0.000000 -0.001218   

InstrumentID  SH600010  SH600011  SH600012  SH600015  ...  SZ301519  SZ301520  \
Date                                                  ...                       
20230103     -0.005181 -0.003611 -0.001650 -0.002121  ...  0.000000  0.000000   
20230104      0.000000 -0.003198 -0.001977 -0.002452  ...  0.000000  0.000000   
20230105      0.000000 -0.003649 -0.001918 -0.002137  ...  0.000000  0.000000   
20230106      0.000000 -0.003019 -0.001613 -0.003166  ...  0.000000  0.000000   
20230109     -0.010152 -0.004065 -0.001886 -0.002235  ...  0.000000  0.000000   
...                ...       ...       ...       ...  ...       ...       ...   
20230928     -0.005814 -0.002252 -0.002329 -0.002183  ... -0.001507 -0.013727   
20231009     -0.005814 -0.001997 -0.002192 -0.002453  ... -0.002317 -0.006879   
20231010     -0.006389 -0.002566 -0.001931 -0.002349  ... -0.001713 -0.005204   
20231011      0.000000 -0.002166 -0.001851 -0.001780  ... -0.002451 -0.003205   
20231012     -0.005814 -0.002023 -0.002354 -0.001918  ... -0.001769 -0.004534   

InstrumentID  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  SZ301550  \
Date                                                                       
20230103      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230104      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230105      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230106      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230109      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...                ...       ...       ...       ...       ...       ...   
20230928     -0.003027 -0.001489 -0.002656 -0.002220 -0.002849 -0.006299   
20231009     -0.002496 -0.001717 -0.003965 -0.002224 -0.002927 -0.005438   
20231010     -0.003523 -0.002025 -0.003561 -0.002199 -0.002106 -0.010185   
20231011     -0.002451 -0.002060 -0.002938 -0.002560 -0.001851 -0.009539   
20231012     -0.003812 -0.002040 -0.003320 -0.001753 -0.002675 -0.006468   

InstrumentID  SZ301558  SZ301559  
Date                              
20230103      0.000000  0.000000  
20230104      0.000000  0.000000  
20230105      0.000000  0.000000  
20230106      0.000000  0.000000  
20230109      0.000000  0.000000  
...                ...       ...  
20230928     -0.017631  0.000000  
20231009     -0.012245  0.000000  
20231010     -0.008934  0.000000  
20231011     -0.007104 -0.009819  
20231012     -0.006352 -0.004899  

[186 rows x 5111 columns]

In [9]:
eg = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/interday_cvar_neg_0.9_21_22_min.csv')
eg

Date  SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
0    20210104 -0.002489 -0.002604 -0.005117 -0.002957 -0.006110 -0.003075   
1    20210105 -0.001675 -0.002426 -0.006855 -0.001922 -0.004014 -0.002728   
2    20210106 -0.001678 -0.002765 -0.005206 -0.002439 -0.003835 -0.002591   
3    20210107 -0.002400 -0.003025 -0.006216 -0.003578 -0.003906 -0.002387   
4    20210108 -0.002419 -0.002861 -0.005919 -0.002562 -0.003690 -0.002498   
..        ...       ...       ...       ...       ...       ...       ...   
480  20221226 -0.001926 -0.003213 -0.002112 -0.001891 -0.003597 -0.002076   
481  20221227 -0.001393 -0.004508 -0.002353 -0.002238 -0.003559 -0.003293   
482  20221228 -0.001651 -0.003081 -0.004029 -0.002394 -0.007092 -0.002061   
483  20221229 -0.002016 -0.003311 -0.002434 -0.003208 -0.003547 -0.002427   
484  20221230 -0.001878 -0.002391 -0.002044 -0.002020 -0.003546 -0.001788   

     SH600010  SH600011  SH600012  ...  SZ301368  SZ301369  SZ301377  \
0   -0.008547 -0.003470 -0.002889  ...       NaN       NaN       NaN   
1    0.000000 -0.003102 -0.003773  ...       NaN       NaN       NaN   
2    0.000000 -0.002888 -0.004765  ...       NaN       NaN       NaN   
3   -0.008547 -0.003338 -0.003229  ...       NaN       NaN       NaN   
4    0.000000 -0.003215 -0.004016  ...       NaN       NaN       NaN   
..        ...       ...       ...  ...       ...       ...       ...   
480 -0.005263 -0.002744 -0.002481  ... -0.001966 -0.001633 -0.001926   
481 -0.005181 -0.002681 -0.002079  ... -0.002333 -0.002068 -0.002444   
482 -0.005102 -0.004060 -0.001918  ... -0.002217 -0.001956 -0.001724   
483 -0.007175 -0.003241 -0.002086  ... -0.001905 -0.002593 -0.002153   
484  0.000000 -0.003053 -0.001749  ... -0.002144 -0.001774 -0.002176   

     SZ301379  SZ301380  SZ301388  SZ301389  SZ301391  SZ301396  SZ301398  
0         NaN       NaN       NaN       NaN       NaN       NaN       NaN  
1         NaN       NaN       NaN       NaN       NaN       NaN       NaN  
2         NaN       NaN       NaN       NaN       NaN       NaN       NaN  
3         NaN       NaN       NaN       NaN       NaN       NaN       NaN  
4         NaN       NaN       NaN       NaN       NaN       NaN       NaN  
..        ...       ...       ...       ...       ...       ...       ...  
480 -0.001597 -0.002441 -0.002123 -0.001597 -0.004109 -0.002208 -0.003076  
481 -0.001802 -0.002658 -0.002192 -0.002407 -0.003983 -0.002846 -0.004253  
482 -0.001755 -0.003459 -0.001928 -0.001990 -0.003002 -0.002241 -0.003321  
483 -0.001868 -0.002145 -0.002203 -0.001720 -0.002629 -0.002447 -0.002642  
484 -0.001623 -0.003071 -0.001039 -0.001540 -0.003185 -0.002043 -0.005474  

[485 rows x 4970 columns]

In [12]:
# eg.set_index('Date', inplace=True)
eg

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                   
20210104 -0.002489 -0.002604 -0.005117 -0.002957 -0.006110 -0.003075   
20210105 -0.001675 -0.002426 -0.006855 -0.001922 -0.004014 -0.002728   
20210106 -0.001678 -0.002765 -0.005206 -0.002439 -0.003835 -0.002591   
20210107 -0.002400 -0.003025 -0.006216 -0.003578 -0.003906 -0.002387   
20210108 -0.002419 -0.002861 -0.005919 -0.002562 -0.003690 -0.002498   
...            ...       ...       ...       ...       ...       ...   
20221226 -0.001926 -0.003213 -0.002112 -0.001891 -0.003597 -0.002076   
20221227 -0.001393 -0.004508 -0.002353 -0.002238 -0.003559 -0.003293   
20221228 -0.001651 -0.003081 -0.004029 -0.002394 -0.007092 -0.002061   
20221229 -0.002016 -0.003311 -0.002434 -0.003208 -0.003547 -0.002427   
20221230 -0.001878 -0.002391 -0.002044 -0.002020 -0.003546 -0.001788   

          SH600010  SH600011  SH600012  SH600015  ...  SZ301368  SZ301369  \
Date                                              ...                       
20210104 -0.008547 -0.003470 -0.002889 -0.002897  ...       NaN       NaN   
20210105  0.000000 -0.003102 -0.003773 -0.002218  ...       NaN       NaN   
20210106  0.000000 -0.002888 -0.004765 -0.001840  ...       NaN       NaN   
20210107 -0.008547 -0.003338 -0.003229 -0.002272  ...       NaN       NaN   
20210108  0.000000 -0.003215 -0.004016 -0.002475  ...       NaN       NaN   
...            ...       ...       ...       ...  ...       ...       ...   
20221226 -0.005263 -0.002744 -0.002481 -0.001953  ... -0.001966 -0.001633   
20221227 -0.005181 -0.002681 -0.002079 -0.002030  ... -0.002333 -0.002068   
20221228 -0.005102 -0.004060 -0.001918 -0.002253  ... -0.002217 -0.001956   
20221229 -0.007175 -0.003241 -0.002086 -0.002357  ... -0.001905 -0.002593   
20221230  0.000000 -0.003053 -0.001749 -0.002230  ... -0.002144 -0.001774   

          SZ301377  SZ301379  SZ301380  SZ301388  SZ301389  SZ301391  \
Date                                                                   
20210104       NaN       NaN       NaN       NaN       NaN       NaN   
20210105       NaN       NaN       NaN       NaN       NaN       NaN   
20210106       NaN       NaN       NaN       NaN       NaN       NaN   
20210107       NaN       NaN       NaN       NaN       NaN       NaN   
20210108       NaN       NaN       NaN       NaN       NaN       NaN   
...            ...       ...       ...       ...       ...       ...   
20221226 -0.001926 -0.001597 -0.002441 -0.002123 -0.001597 -0.004109   
20221227 -0.002444 -0.001802 -0.002658 -0.002192 -0.002407 -0.003983   
20221228 -0.001724 -0.001755 -0.003459 -0.001928 -0.001990 -0.003002   
20221229 -0.002153 -0.001868 -0.002145 -0.002203 -0.001720 -0.002629   
20221230 -0.002176 -0.001623 -0.003071 -0.001039 -0.001540 -0.003185   

          SZ301396  SZ301398  
Date                          
20210104       NaN       NaN  
20210105       NaN       NaN  
20210106       NaN       NaN  
20210107       NaN       NaN  
20210108       NaN       NaN  
...            ...       ...  
20221226 -0.002208 -0.003076  
20221227 -0.002846 -0.004253  
20221228 -0.002241 -0.003321  
20221229 -0.002447 -0.002642  
20221230 -0.002043 -0.005474  

[485 rows x 4969 columns]

In [13]:
cvar = pd.concat([eg, cvar_neg])
cvar

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                   
20210104 -0.002489 -0.002604 -0.005117 -0.002957 -0.006110 -0.003075   
20210105 -0.001675 -0.002426 -0.006855 -0.001922 -0.004014 -0.002728   
20210106 -0.001678 -0.002765 -0.005206 -0.002439 -0.003835 -0.002591   
20210107 -0.002400 -0.003025 -0.006216 -0.003578 -0.003906 -0.002387   
20210108 -0.002419 -0.002861 -0.005919 -0.002562 -0.003690 -0.002498   
...            ...       ...       ...       ...       ...       ...   
20230928 -0.001666 -0.001611 -0.001812 -0.001356  0.000000 -0.001129   
20231009 -0.001954 -0.002545 -0.002074 -0.002048 -0.003546 -0.001756   
20231010 -0.002128 -0.001681 -0.002172 -0.001428 -0.003559 -0.001214   
20231011 -0.001973 -0.001394 -0.002257 -0.001707 -0.003559 -0.001009   
20231012 -0.001901 -0.001124 -0.002033 -0.001481  0.000000 -0.001218   

          SH600010  SH600011  SH600012  SH600015  ...  SZ301519  SZ301520  \
Date                                              ...                       
20210104 -0.008547 -0.003470 -0.002889 -0.002897  ...       NaN       NaN   
20210105  0.000000 -0.003102 -0.003773 -0.002218  ...       NaN       NaN   
20210106  0.000000 -0.002888 -0.004765 -0.001840  ...       NaN       NaN   
20210107 -0.008547 -0.003338 -0.003229 -0.002272  ...       NaN       NaN   
20210108  0.000000 -0.003215 -0.004016 -0.002475  ...       NaN       NaN   
...            ...       ...       ...       ...  ...       ...       ...   
20230928 -0.005814 -0.002252 -0.002329 -0.002183  ... -0.001507 -0.013727   
20231009 -0.005814 -0.001997 -0.002192 -0.002453  ... -0.002317 -0.006879   
20231010 -0.006389 -0.002566 -0.001931 -0.002349  ... -0.001713 -0.005204   
20231011  0.000000 -0.002166 -0.001851 -0.001780  ... -0.002451 -0.003205   
20231012 -0.005814 -0.002023 -0.002354 -0.001918  ... -0.001769 -0.004534   

          SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  SZ301550  \
Date                                                                   
20210104       NaN       NaN       NaN       NaN       NaN       NaN   
20210105       NaN       NaN       NaN       NaN       NaN       NaN   
20210106       NaN       NaN       NaN       NaN       NaN       NaN   
20210107       NaN       NaN       NaN       NaN       NaN       NaN   
20210108       NaN       NaN       NaN       NaN       NaN       NaN   
...            ...       ...       ...       ...       ...       ...   
20230928 -0.003027 -0.001489 -0.002656 -0.002220 -0.002849 -0.006299   
20231009 -0.002496 -0.001717 -0.003965 -0.002224 -0.002927 -0.005438   
20231010 -0.003523 -0.002025 -0.003561 -0.002199 -0.002106 -0.010185   
20231011 -0.002451 -0.002060 -0.002938 -0.002560 -0.001851 -0.009539   
20231012 -0.003812 -0.002040 -0.003320 -0.001753 -0.002675 -0.006468   

          SZ301558  SZ301559  
Date                          
20210104       NaN       NaN  
20210105       NaN       NaN  
20210106       NaN       NaN  
20210107       NaN       NaN  
20210108       NaN       NaN  
...            ...       ...  
20230928 -0.017631  0.000000  
20231009 -0.012245  0.000000  
20231010 -0.008934  0.000000  
20231011 -0.007104 -0.009819  
20231012 -0.006352 -0.004899  

[671 rows x 5175 columns]

In [14]:
cvar.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/interday_cvar_neg_0.9_21_23_min.csv')

In [15]:
def conti_up_down_v2(price: pd.DataFrame, T=30, is_up=True):
    '''
    返回日内连续性涨跌幅,使用卷积计算,
    price:Multi-index对象,
    T:最大回溯窗口期,
    is_up:True时计算涨幅, False时计算跌幅
    '''

    kernel_arrays = [np.ones(i) for i in range(1, T+1)]

    def _conti_up_down(price_daily, T=T, is_up=is_up):
        return_daily = price_daily.pct_change().fillna(0)

        def _for_stock(r_daily_stock):
            r_mat = np.vstack([
                np.pad(np.convolve(r_daily_stock, kernel_arrays[i], mode='valid'), (i, 0), mode='constant') for i in range(len(kernel_arrays))
                ])
            
            if is_up:
                res = np.max(r_mat)
            else:
                res = np.min(r_mat)
            
            return res
        
        return return_daily.apply(_for_stock)
    
    return price.groupby(level='Date').progress_apply(_conti_up_down)

In [16]:
conti2023 = conti_up_down_v2(open2023)
conti2023

InstrumentID  SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                       
20230103      0.006941  0.013903  0.012613  0.032589  0.010626  0.010241   
20230104      0.008251  0.014473  0.010673  0.020436  0.007017  0.012138   
20230105      0.006851  0.030577  0.005316  0.015302  0.007052  0.032643   
20230106      0.005483  0.008921  0.014085  0.014926  0.007066  0.011311   
20230109      0.008206  0.021531  0.015817  0.012501  0.007029  0.018966   
...                ...       ...       ...       ...       ...       ...   
20230928      0.004213  0.005402  0.012704  0.008765  0.007129  0.009286   
20231009      0.004288  0.007447  0.010686  0.007764  0.007142  0.005393   
20231010      0.004261  0.010179  0.014057  0.008465  0.007168  0.013157   
20231011      0.007106  0.008409  0.005285  0.007864  0.007206  0.003496   
20231012      0.009841  0.006491  0.017628  0.004719  0.010777  0.008324   

InstrumentID  SH600010  SH600011  SH600012  SH600015  ...  SZ301519  SZ301520  \
Date                                                  ...                       
20230103      0.010524  0.032458  0.010933  0.007748  ...  0.000000  0.000000   
20230104      0.010576  0.017880  0.009596  0.011483  ...  0.000000  0.000000   
20230105      0.010469  0.021557  0.008141  0.005729  ...  0.000000  0.000000   
20230106      0.010576  0.031750  0.004085  0.005732  ...  0.000000  0.000000   
20230109      0.020699  0.021549  0.020316  0.007641  ...  0.000000  0.000000   
...                ...       ...       ...       ...  ...       ...       ...   
20230928      0.011865  0.006366  0.017150  0.007005  ...  0.012386  0.044090   
20231009      0.011831  0.010225  0.010858  0.008878  ...  0.010408  0.065111   
20231010      0.011730  0.005173  0.005472  0.007070  ...  0.014364  0.016877   
20231011      0.011934  0.010314  0.009289  0.008906  ...  0.019802  0.047251   
20231012      0.011897  0.015615  0.021123  0.008818  ...  0.012729  0.037963   

InstrumentID  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  SZ301550  \
Date                                                                       
20230103      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230104      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230105      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230106      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230109      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...                ...       ...       ...       ...       ...       ...   
20230928      0.011849  0.011068  0.025191  0.011515  0.022158  0.057552   
20231009      0.015172  0.007507  0.015687  0.011705  0.015883  0.037972   
20231010      0.012883  0.024354  0.030568  0.010891  0.009121  0.142583   
20231011      0.019749  0.026271  0.024716  0.023816  0.018498  0.062561   
20231012      0.023885  0.022519  0.015044  0.012880  0.014590  0.047592   

InstrumentID  SZ301558  SZ301559  
Date                              
20230103      0.000000  0.000000  
20230104      0.000000  0.000000  
20230105      0.000000  0.000000  
20230106      0.000000  0.000000  
20230109      0.000000  0.000000  
...                ...       ...  
20230928      0.116767  0.000000  
20231009      0.036390  0.000000  
20231010      0.101802  0.000000  
20231011      0.085532  0.061471  
20231012      0.036988  0.031350  

[186 rows x 5111 columns]

In [17]:
conti_21_22 = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/conti_up_T30_21_22_min.csv')
conti_21_22.set_index('Date', inplace=True)
conti_21_22

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                   
20210104  0.012475  0.020778  0.068378  0.009657  0.010673  0.042181   
20210105  0.014608  0.016758  0.022777  0.010455  0.010625  0.013094   
20210106  0.013452  0.020484  0.018303  0.018121  0.007129  0.030760   
20210107  0.014392  0.009766  0.045284  0.019629  0.007206  0.011407   
20210108  0.012257  0.037796  0.033847  0.013061  0.021391  0.019230   
...            ...       ...       ...       ...       ...       ...   
20221226  0.005585  0.034904  0.014063  0.012284  0.007285  0.026296   
20221227  0.012467  0.017935  0.008793  0.012376  0.007232  0.016450   
20221228  0.008266  0.012217  0.005367  0.016307  0.014361  0.014010   
20221229  0.009728  0.018562  0.014213  0.017512  0.010739  0.010584   
20221230  0.006925  0.011436  0.007089  0.012773  0.010726  0.010295   

          SH600010  SH600011  SH600012  SH600015  ...  SZ301368  SZ301369  \
Date                                              ...                       
20210104  0.017753  0.017833  0.014728  0.008109  ...       NaN       NaN   
20210105  0.017536  0.006896  0.023329  0.019332  ...       NaN       NaN   
20210106  0.009210  0.018307  0.052855  0.009608  ...       NaN       NaN   
20210107  0.026084  0.020460  0.014560  0.009690  ...       NaN       NaN   
20210108  0.009210  0.020455  0.028044  0.011233  ...       NaN       NaN   
...            ...       ...       ...       ...  ...       ...       ...   
20221226  0.015845  0.024941  0.012790  0.005871  ...  0.015319  0.021112   
20221227  0.020723  0.014374  0.013972  0.009720  ...  0.009936  0.036444   
20221228  0.010440  0.049185  0.009693  0.013517  ...  0.012101  0.007957   
20221229  0.010442  0.016395  0.011148  0.009750  ...  0.025627  0.014225   
20221230  0.005454  0.022638  0.012464  0.007741  ...  0.018582  0.004325   

          SZ301377  SZ301379  SZ301380  SZ301388  SZ301389  SZ301391  \
Date                                                                   
20210104       NaN       NaN       NaN       NaN       NaN       NaN   
20210105       NaN       NaN       NaN       NaN       NaN       NaN   
20210106       NaN       NaN       NaN       NaN       NaN       NaN   
20210107       NaN       NaN       NaN       NaN       NaN       NaN   
20210108       NaN       NaN       NaN       NaN       NaN       NaN   
...            ...       ...       ...       ...       ...       ...   
20221226  0.021364  0.035990  0.041000  0.057451  0.016254  0.017476   
20221227  0.013988  0.008304  0.013344  0.011615  0.029674  0.017099   
20221228  0.005562  0.007288  0.016924  0.007530  0.007055  0.014703   
20221229  0.016630  0.015949  0.021080  0.008458  0.011953  0.013990   
20221230  0.009652  0.009371  0.039320  0.008549  0.006546  0.023961   

          SZ301396  SZ301398  
Date                          
20210104       NaN       NaN  
20210105       NaN       NaN  
20210106       NaN       NaN  
20210107       NaN       NaN  
20210108       NaN       NaN  
...            ...       ...  
20221226  0.012711  0.021967  
20221227  0.009558  0.017285  
20221228  0.009360  0.010535  
20221229  0.019438  0.030464  
20221230  0.019521  0.063845  

[485 rows x 4969 columns]

In [20]:
conti_21_23 = pd.concat([conti_21_22, conti2023])
conti_21_23

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                   
20210104  0.012475  0.020778  0.068378  0.009657  0.010673  0.042181   
20210105  0.014608  0.016758  0.022777  0.010455  0.010625  0.013094   
20210106  0.013452  0.020484  0.018303  0.018121  0.007129  0.030760   
20210107  0.014392  0.009766  0.045284  0.019629  0.007206  0.011407   
20210108  0.012257  0.037796  0.033847  0.013061  0.021391  0.019230   
...            ...       ...       ...       ...       ...       ...   
20230928  0.004213  0.005402  0.012704  0.008765  0.007129  0.009286   
20231009  0.004288  0.007447  0.010686  0.007764  0.007142  0.005393   
20231010  0.004261  0.010179  0.014057  0.008465  0.007168  0.013157   
20231011  0.007106  0.008409  0.005285  0.007864  0.007206  0.003496   
20231012  0.009841  0.006491  0.017628  0.004719  0.010777  0.008324   

          SH600010  SH600011  SH600012  SH600015  ...  SZ301519  SZ301520  \
Date                                              ...                       
20210104  0.017753  0.017833  0.014728  0.008109  ...       NaN       NaN   
20210105  0.017536  0.006896  0.023329  0.019332  ...       NaN       NaN   
20210106  0.009210  0.018307  0.052855  0.009608  ...       NaN       NaN   
20210107  0.026084  0.020460  0.014560  0.009690  ...       NaN       NaN   
20210108  0.009210  0.020455  0.028044  0.011233  ...       NaN       NaN   
...            ...       ...       ...       ...  ...       ...       ...   
20230928  0.011865  0.006366  0.017150  0.007005  ...  0.012386  0.044090   
20231009  0.011831  0.010225  0.010858  0.008878  ...  0.010408  0.065111   
20231010  0.011730  0.005173  0.005472  0.007070  ...  0.014364  0.016877   
20231011  0.011934  0.010314  0.009289  0.008906  ...  0.019802  0.047251   
20231012  0.011897  0.015615  0.021123  0.008818  ...  0.012729  0.037963   

          SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  SZ301550  \
Date                                                                   
20210104       NaN       NaN       NaN       NaN       NaN       NaN   
20210105       NaN       NaN       NaN       NaN       NaN       NaN   
20210106       NaN       NaN       NaN       NaN       NaN       NaN   
20210107       NaN       NaN       NaN       NaN       NaN       NaN   
20210108       NaN       NaN       NaN       NaN       NaN       NaN   
...            ...       ...       ...       ...       ...       ...   
20230928  0.011849  0.011068  0.025191  0.011515  0.022158  0.057552   
20231009  0.015172  0.007507  0.015687  0.011705  0.015883  0.037972   
20231010  0.012883  0.024354  0.030568  0.010891  0.009121  0.142583   
20231011  0.019749  0.026271  0.024716  0.023816  0.018498  0.062561   
20231012  0.023885  0.022519  0.015044  0.012880  0.014590  0.047592   

          SZ301558  SZ301559  
Date                          
20210104       NaN       NaN  
20210105       NaN       NaN  
20210106       NaN       NaN  
20210107       NaN       NaN  
20210108       NaN       NaN  
...            ...       ...  
20230928  0.116767  0.000000  
20231009  0.036390  0.000000  
20231010  0.101802  0.000000  
20231011  0.085532  0.061471  
20231012  0.036988  0.031350  

[671 rows x 5175 columns]

In [21]:
def long_short_return_v2(df:pd.DataFrame, window=15):
    '''
    日内单侧行情, 衡量多空博弈力量, 判断急涨急跌
    '''
    def _long_short_return(data:pd.DataFrame):
        data = data.pct_change().fillna(0)
        data_mean_sum = data.rolling(window).sum().fillna(0)
        return data_mean_sum.max() - data_mean_sum.min()
    
    return df.groupby(level='Date').progress_apply(_long_short_return)

In [22]:
ls_23 = long_short_return_v2(open2023)
ls_23

InstrumentID  SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                       
20230103      0.017956  0.029867  0.012570  0.036097  0.010626  0.016807   
20230104      0.009572  0.017932  0.015973  0.025311  0.013829  0.018912   
20230105      0.013628  0.038536  0.015827  0.020668  0.013853  0.034435   
20230106      0.010901  0.023189  0.015808  0.034403  0.013950  0.028633   
20230109      0.014951  0.022745  0.017449  0.023614  0.013913  0.020244   
...                ...       ...       ...       ...       ...       ...   
20230928      0.007005  0.013394  0.014447  0.012354  0.010576  0.013297   
20231009      0.014170  0.030867  0.014232  0.009886  0.014159  0.024790   
20231010      0.014113  0.013891  0.019329  0.013231  0.014172  0.015814   
20231011      0.009935  0.015798  0.014011  0.012090  0.010727  0.008076   
20231012      0.014014  0.009245  0.022860  0.007861  0.014222  0.012858   

InstrumentID  SH600010  SH600011  SH600012  SH600015  ...  SZ301519  SZ301520  \
Date                                                  ...                       
20230103      0.020752  0.024643  0.012336  0.013499  ...  0.000000  0.000000   
20230104      0.020699  0.039406  0.013637  0.009546  ...  0.000000  0.000000   
20230105      0.020619  0.037954  0.010825  0.009491  ...  0.000000  0.000000   
20230106      0.020673  0.032622  0.006797  0.009502  ...  0.000000  0.000000   
20230109      0.025800  0.043332  0.020194  0.013326  ...  0.000000  0.000000   
...                ...       ...       ...       ...  ...       ...       ...   
20230928      0.023257  0.015219  0.020689  0.010468  ...  0.012788  0.098888   
20231009      0.011729  0.012688  0.020812  0.017649  ...  0.022332  0.088104   
20231010      0.023157  0.021618  0.017216  0.012343  ...  0.018068  0.048634   
20231011      0.017511  0.015571  0.022158  0.012440  ...  0.041048  0.058049   
20231012      0.017476  0.020806  0.028378  0.014017  ...  0.022287  0.055881   

InstrumentID  SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  SZ301550  \
Date                                                                       
20230103      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230104      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230105      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230106      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20230109      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...                ...       ...       ...       ...       ...       ...   
20230928      0.027738  0.014896  0.027419  0.022393  0.033254  0.092560   
20231009      0.022607  0.017905  0.028345  0.031122  0.029222  0.059908   
20231010      0.024895  0.033195  0.039763  0.016670  0.018959  0.194342   
20231011      0.034734  0.035484  0.029187  0.032240  0.025126  0.094899   
20231012      0.039030  0.027233  0.030710  0.021682  0.032863  0.073237   

InstrumentID  SZ301558  SZ301559  
Date                              
20230103      0.000000  0.000000  
20230104      0.000000  0.000000  
20230105      0.000000  0.000000  
20230106      0.000000  0.000000  
20230109      0.000000  0.000000  
...                ...       ...  
20230928      0.152920  0.000000  
20231009      0.102366  0.000000  
20231010      0.105289  0.000000  
20231011      0.097099  0.100193  
20231012      0.064816  0.047041  

[186 rows x 5111 columns]

In [23]:
ls_21_22 = pd.read_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/longshort_v3_vol_ret_w15_21_22_min.csv')
ls_21_22.set_index('Date', inplace=True)
ls_21_22

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                   
20210104  0.000284  0.000372  0.000957  0.000211  0.000252  0.000649   
20210105  0.000240  0.000236  0.001196  0.000136  0.000306  0.000140   
20210106  0.000199  0.000199  0.000508  0.000544  0.000160  0.000497   
20210107  0.000339  0.000205  0.001019  0.004656  0.000408  0.000173   
20210108  0.000219  0.000457  0.000653  0.000341  0.001480  0.000370   
...            ...       ...       ...       ...       ...       ...   
20221226  0.000192  0.000311  0.000301  0.000270  0.000267  0.000203   
20221227  0.000598  0.000644  0.000633  0.000251  0.000370  0.000544   
20221228  0.000195  0.000246  0.000595  0.000302  0.000486  0.000131   
20221229  0.000243  0.000310  0.000294  0.001068  0.001025  0.000306   
20221230  0.000240  0.000174  0.000366  0.000166  0.000796  0.000158   

          SH600010  SH600011  SH600012  SH600015  ...  SZ301368  SZ301369  \
Date                                              ...                       
20210104  0.000691  0.000306  0.000211  0.000324  ...       NaN       NaN   
20210105  0.000691  0.000256  0.000484  0.000464  ...       NaN       NaN   
20210106  0.000547  0.000366  0.000731  0.000397  ...       NaN       NaN   
20210107  0.003068  0.000570  0.000371  0.000152  ...       NaN       NaN   
20210108  0.000326  0.000329  0.000598  0.000231  ...       NaN       NaN   
...            ...       ...       ...       ...  ...       ...       ...   
20221226  0.000815  0.000388  0.000303  0.000160  ...  0.000365  0.000314   
20221227  0.001524  0.000333  0.000400  0.000469  ...  0.000249  0.000696   
20221228  0.000321  0.000918  0.000452  0.000243  ...  0.000507  0.000129   
20221229  0.000615  0.000352  0.000194  0.000351  ...  0.001042  0.000187   
20221230  0.000384  0.000550  0.000164  0.000257  ...  0.000257  0.000140   

          SZ301377  SZ301379  SZ301380  SZ301388  SZ301389  SZ301391  \
Date                                                                   
20210104       NaN       NaN       NaN       NaN       NaN       NaN   
20210105       NaN       NaN       NaN       NaN       NaN       NaN   
20210106       NaN       NaN       NaN       NaN       NaN       NaN   
20210107       NaN       NaN       NaN       NaN       NaN       NaN   
20210108       NaN       NaN       NaN       NaN       NaN       NaN   
...            ...       ...       ...       ...       ...       ...   
20221226  0.000739  0.001696  0.002528  0.003026  0.000263  0.000531   
20221227  0.000293  0.000411  0.000655  0.000287  0.001041  0.000604   
20221228  0.000158  0.000200  0.000547  0.000243  0.000441  0.000380   
20221229  0.000743  0.000313  0.000237  0.000328  0.000213  0.000388   
20221230  0.000349  0.000139  0.000733  0.000162  0.000217  0.000774   

          SZ301396  SZ301398  
Date                          
20210104       NaN       NaN  
20210105       NaN       NaN  
20210106       NaN       NaN  
20210107       NaN       NaN  
20210108       NaN       NaN  
...            ...       ...  
20221226  0.001074  0.000364  
20221227  0.000797  0.000394  
20221228  0.000254  0.000292  
20221229  0.000359  0.000612  
20221230  0.000517  0.001909  

[485 rows x 4969 columns]

In [24]:
ls_21_23 = pd.concat([ls_21_22, ls_23])
ls_21_23

SH600000  SH600004  SH600006  SH600007  SH600008  SH600009  \
Date                                                                   
20210104  0.000284  0.000372  0.000957  0.000211  0.000252  0.000649   
20210105  0.000240  0.000236  0.001196  0.000136  0.000306  0.000140   
20210106  0.000199  0.000199  0.000508  0.000544  0.000160  0.000497   
20210107  0.000339  0.000205  0.001019  0.004656  0.000408  0.000173   
20210108  0.000219  0.000457  0.000653  0.000341  0.001480  0.000370   
...            ...       ...       ...       ...       ...       ...   
20230928  0.007005  0.013394  0.014447  0.012354  0.010576  0.013297   
20231009  0.014170  0.030867  0.014232  0.009886  0.014159  0.024790   
20231010  0.014113  0.013891  0.019329  0.013231  0.014172  0.015814   
20231011  0.009935  0.015798  0.014011  0.012090  0.010727  0.008076   
20231012  0.014014  0.009245  0.022860  0.007861  0.014222  0.012858   

          SH600010  SH600011  SH600012  SH600015  ...  SZ301519  SZ301520  \
Date                                              ...                       
20210104  0.000691  0.000306  0.000211  0.000324  ...       NaN       NaN   
20210105  0.000691  0.000256  0.000484  0.000464  ...       NaN       NaN   
20210106  0.000547  0.000366  0.000731  0.000397  ...       NaN       NaN   
20210107  0.003068  0.000570  0.000371  0.000152  ...       NaN       NaN   
20210108  0.000326  0.000329  0.000598  0.000231  ...       NaN       NaN   
...            ...       ...       ...       ...  ...       ...       ...   
20230928  0.023257  0.015219  0.020689  0.010468  ...  0.012788  0.098888   
20231009  0.011729  0.012688  0.020812  0.017649  ...  0.022332  0.088104   
20231010  0.023157  0.021618  0.017216  0.012343  ...  0.018068  0.048634   
20231011  0.017511  0.015571  0.022158  0.012440  ...  0.041048  0.058049   
20231012  0.017476  0.020806  0.028378  0.014017  ...  0.022287  0.055881   

          SZ301525  SZ301528  SZ301529  SZ301533  SZ301548  SZ301550  \
Date                                                                   
20210104       NaN       NaN       NaN       NaN       NaN       NaN   
20210105       NaN       NaN       NaN       NaN       NaN       NaN   
20210106       NaN       NaN       NaN       NaN       NaN       NaN   
20210107       NaN       NaN       NaN       NaN       NaN       NaN   
20210108       NaN       NaN       NaN       NaN       NaN       NaN   
...            ...       ...       ...       ...       ...       ...   
20230928  0.027738  0.014896  0.027419  0.022393  0.033254  0.092560   
20231009  0.022607  0.017905  0.028345  0.031122  0.029222  0.059908   
20231010  0.024895  0.033195  0.039763  0.016670  0.018959  0.194342   
20231011  0.034734  0.035484  0.029187  0.032240  0.025126  0.094899   
20231012  0.039030  0.027233  0.030710  0.021682  0.032863  0.073237   

          SZ301558  SZ301559  
Date                          
20210104       NaN       NaN  
20210105       NaN       NaN  
20210106       NaN       NaN  
20210107       NaN       NaN  
20210108       NaN       NaN  
...            ...       ...  
20230928  0.152920  0.000000  
20231009  0.102366  0.000000  
20231010  0.105289  0.000000  
20231011  0.097099  0.100193  
20231012  0.064816  0.047041  

[671 rows x 5175 columns]

In [25]:
ls_21_23.to_csv('/mnt/datadisk2/aglv/aglv/lab_aglv/aglv_factor/factor_829/longshort_v3_vol_ret_w15_21_23_min.csv')